In [1]:
import pandas as pd
import numpy as np

In [2]:
# https://www.kaggle.com/datasets/deepcontractor/car-price-prediction-challenge
df = pd.read_csv('data/car_price_prediction.csv')

In [3]:
# rename columns
df.columns = ['car_id', 'price', 'levy', 'manufacturer', 'model', 'prod_year', 'category',
              'leather_interior', 'fuel_type', 'engine_volume', 'mileage', 'cylinders', 'gear_box_type',
              'drive_wheels','doors', 'wheel', 'color', 'airbags']

In [4]:
# i'm including tesla in this case b/c there's one tesla car in the dataset that's model X and worth $53K
luxury_brands = ['LEXUS', 'MERCEDES-BENZ', 'PORSCHE', 'BMW', 'AUDI', 'INFINITI',
                'ALFA ROMEO', 'ACURA', 'LINCOLN', 'LAND ROVER', 'JAGUAR', 'CADILLAC',
                'BENTLEY', 'VOLVO', 'MASERATI', 'FERRARI', 'LAMBORGHINI', 'ROLLS-ROYCE',
                'ASTON MARTIN', 'TESLA']

df['is_luxury_manufacturer'] = df['manufacturer'].apply(lambda x: x in luxury_brands)

In [5]:
# map random colors to reduce the list
color_mapping = {'Carnelian red': 'Red',
                 'Beige': 'Brown',
                 'Golden': 'Yellow',
                 'Pink': 'Other',
                 'Green': 'Other',
                 'Purple': 'Other',
                 'Orange': 'Other',
                 'Sky blue': 'Blue'}

df['color_new'] = df['color'].map(color_mapping).fillna(df['color'])

In [6]:
# the original mileage column had km recorded and was in km. converting to miles
df['mileage_miles'] = df['mileage'].str.replace(' km', '').astype(int) * 0.621371

In [7]:
# assume this data was collected in 2021 and prices were listed as 2021.. or assume current year is 2021
df['car_age'] = 2021 - df.prod_year

In [8]:
# creating a capped price column out of curiosity
percentile_97_5 = np.percentile(df['price'], 97.5)
df['capped_price'] = df['price'].clip(upper=percentile_97_5)

In [9]:
# convert engine volume to 2 columns: engine volume new and if it's turbo
df['engine_volume_new'] = df['engine_volume'].str.replace(' Turbo', '').astype(float)
df['turbo_engine_ind'] = df['engine_volume'].str.contains('Turbo')

In [10]:
# map gear box type to automatic transmission indicator
df['is_automatic_transmission_ind'] = df['gear_box_type'].map({'Automatic': 1, 'Tiptronic': 1, 'Variator': 1, 'Manual': 0})

In [11]:
# converting categorical columns
df = pd.get_dummies(df, columns=['category'], drop_first=True)

In [12]:
df = pd.get_dummies(df, columns=['fuel_type'], drop_first=True)

In [13]:
df = pd.get_dummies(df, columns=['gear_box_type'], drop_first=True)

In [14]:
df = pd.get_dummies(df, columns=['drive_wheels'], drop_first=True)

In [15]:
df = pd.get_dummies(df, columns=['color_new'], drop_first=True)

In [16]:
# map leather interior to True False
df['leather_interior'] = df['leather_interior'].map({'Yes': 1, 'No': 0})

In [17]:
percentile_99_5 = np.percentile(df['price'],99.5)

# decided to drop records sold below $1k and past the 99.5th percentile
df_filtered = df[df.price > 1000]
df_filtered = df_filtered[df_filtered.price <= percentile_99_5]

In [18]:
# also decide to remove these records with 0 miles
df_filtered = df_filtered[df_filtered.mileage_miles != 0]

In [19]:
df_filtered.to_csv('data/car_price_prediction_clean_notebook.csv', index = False)